In [ ]:
import numpy as np

# MDP parameters
GRID_SIZE = 3 
R_X = R_Y = 0  

# Define action symbols
action_symbols = ['↑', '↓', '→', '←']  # Symbols for the four possible actions

actions = action_symbols
gamma = 0.99

transition_probs = {'↑': [0.8, 0.0, 0.1, 0.1],
                    '↓': [0, 0.8, 0.1, 0.1],
                    '→': [0.1, 0.1, 0.8, 0],
                    '←': [0.1, 0.1, 0, 0.8]}

rewards = np.array([[0, -1, 10],
                    [-1, -1, -1],
                    [-1, -1, -1]])

value_function = np.zeros((GRID_SIZE, GRID_SIZE))
policy = np.zeros((GRID_SIZE, GRID_SIZE), dtype=str)

In [4]:
def get_new_position(i, j, action):
    if action == '↑':
        return max(i - 1, 0), j
    elif action == '↓':
        return min(i + 1, GRID_SIZE - 1), j
    elif action == '→':
        return i, min(j + 1, GRID_SIZE - 1)
    elif action == '←':
        return i, max(j - 1, 0)

In [19]:
def valid_action(i, j, action):
    match action:
        case '↑':
            return i > 0
        case '↓':
            return i < GRID_SIZE - 1
        case '→':
            return j < GRID_SIZE - 1
        case '←':
            return j > 0
        case _:
            return False  # In case of an unknown action


In [20]:
def value_iteration():
    theta = 0.001
    iteration_count = 0

    while True:
        delta = 0

        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if rewards[i][j] in {10, rewards[R_X][R_Y]}:  # Skip terminal states
                    continue

                old_value = value_function[i][j]
                best_value = float('-inf')
                best_action = None

                for action in actions:
                    if not valid_action(i, j, action):
                        continue

                    expected_value = rewards[i][j]
                    for k, prob in enumerate(transition_probs[action]):
                        ni, nj = get_new_position(i, j, action_symbols[k])
                        expected_value += prob * gamma * value_function[ni][nj]

                    if expected_value > best_value:
                        best_value = expected_value
                        best_action = action

                value_function[i][j] = best_value
                policy[i][j] = best_action
                delta = max(delta, abs(old_value - best_value))

        iteration_count += 1
        if delta < theta:
            break

    return iteration_count


In [21]:
def run_value_iteration(reward_value):

    global rewards
    rewards[R_X][R_Y] = reward_value
    return value_iteration()

In [23]:
r_values = [100, 3, 0, -3]
for r in r_values:
    value_function[R_X, R_Y] = r
    value_function[0, -1] = 10


    num_iterations = run_value_iteration(r)

    print(f"\nOptimal Value Function for r = {r} (Converged in {num_iterations} iterations):")
    print(value_function)
    policy[R_X, R_Y] = 'r'
    policy[0,-1] = 'T'
    print(f"\nOptimal Policy for r = {r}")
    print(policy)


Optimal Value Function for r = 100 (Converged in 13 iterations):
[[100.          97.20351929  10.        ]
 [ 97.20351929  94.75126763  88.20401924]
 [ 94.48181445  92.35290449  89.76213557]]

Optimal Policy for r = 100
[['r' '←' 'T']
 ['↑' '←' '↓']
 ['↑' '←' '←']]

Optimal Value Function for r = 3 (Converged in 41 iterations):
[[ 3.          8.46194904 10.        ]
 [ 5.38366785  7.11322545  8.46194227]
 [ 4.57497057  5.79414801  6.96501766]]

Optimal Policy for r = 3
[['r' '→' 'T']
 ['→' '→' '↑']
 ['→' '→' '↑']]

Optimal Value Function for r = 0 (Converged in 3 iterations):
[[ 0.          8.46193956 10.        ]
 [ 5.08334043  7.1132055   8.46193936]
 [ 4.54187018  5.79411233  6.96500905]]

Optimal Policy for r = 0
[['r' '→' 'T']
 ['→' '→' '↑']
 ['→' '→' '↑']]

Optimal Value Function for r = -3 (Converged in 3 iterations):
[[-3.          8.46193928 10.        ]
 [ 4.78307131  7.11320493  8.46193928]
 [ 4.50887324  5.79411129  6.9650088 ]]

Optimal Policy for r = -3
[['r' '→' 'T']
 [